In [1]:
import sqlite3
import re
from pathlib import Path
import csv
import sys

## check tables structure

In [2]:
!head data/metadata.csv
!head data/genstudio.csv

,dna_chip_id,breed,sex
0,202290551164R09C01,Д,Хр
1,202341831114R02C01,Д,Хр
2,202341831114R03C01,Д,Хр
3,202341831114R04C01,Д,Хр
4,202290551140R01C01,Д,Хр
5,202341831114R05C01,Д,Хр
6,202290551140R03C01,Д,Хр
7,202290551140R05C01,Д,Хр
8,202341831114R07C01,Д,Хр
,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,-,-,1,10573221,0.0,0.0,0.942,0.413,1.0,0.40399999999999997
1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0,0.3017
2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0,0.0388
3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0,0.2761
4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.867,0.983,1.122,0.9994,0.0022
5,1_11426075,6,0,2023418311

## create lists with table build instructions 

In [48]:
base_path = "/home/aither/institute-for-bioinformatics/python2021/BI_2021_Python/"
csv_files_names = {
    f"{base_path}metadata.csv": ["CREATE TABLE IF NOT EXISTS meta(",
            "ind INTEGER,",
            "dna_chip_id TEXT PRIMARY KEY,",
            "breed TEXT,",
            "sex TEXT)"],
    f"{base_path}genstudio.csv": ["CREATE TABLE IF NOT EXISTS genstudio(",
                "ind INTEGER PRIMARY KEY,",
                "SNP_name TEXT,",
                "SNP_Index INTEGER,",
                "SNP_Aux INTEGER,",
                "Sample_ID TEXT NOT NULL REFERENCES meta(dna_chip_id),",
                "SNP TEXT,",
                "Allele1_Top TEXT,",
                "Allele2_Top TEXT,",
                "Allele1_Forward TEXT,",
                "Allele2_Forward TEXT,",
                "Allele1_AB TEXT,",
                "Allele2_AB TEXT,",
                "Chr INTEGER,",
                "Position INTEGER,",
                "GC_Score REAL,",
                "GT_Score REAL,",
                "Theta REAL,",
                "R REAL,",
                "B_Allele_Freq REAL,",
                  "Log_R_Ratio REAL)",        
    ]
}

## functions to fill db with table data

In [93]:
def create_table(tables, db, connection):
    for _, query in tables.items():        
        connection.execute("".join(query))
    connection.commit


def get_columns(query):
    columns = []
    for _, column in enumerate(query[1:]):
        columns.append(column.split(' ')[0])
    return columns


def fill_table(query, values, connection):
    sub_values = ('?, ' * (len(query)-1))[0:-2]
    match = re.search(r"\w+(?=\()", query[0])
    if match:
        table_name = match.group(0)
        columns = ", ".join(get_columns(query))
        sql_code = f"INSERT INTO {table_name}({columns}) VALUES ({sub_values})"
        try:
            if len(values)>1:
                connection.executemany(sql_code, (values))
            else:
                connection.execute(sql_code, (values))
            connection.commit()
        except sqlite3.IntegrityError:
            print("Unexpected error:", sys.exc_info()[0])


def csv_to_db(csv_path_tables, db_name, delimiter=","):
    connection = sqlite3.connect(db_name)
    connection.execute("PRAGMA foreign_keys = ON;")
    create_table(csv_path_tables, db_name, connection)
    for csv_path, table_query in csv_path_tables.items():
        if Path(csv_path).is_file():
            with open(csv_path, 'r') as f:
                content = csv.reader(f, delimiter=delimiter)
                batch = []
                for i, row in enumerate(content):
                    if i:
                        batch.append(row)
                        if len(batch) == 50000:
                            fill_table(table_query, batch, connection)
                            batch = []
                if batch:
                    fill_table(table_query, batch, connection)
    connection.close()

In [92]:
%%timeit 
csv_to_db(csv_files_names, "genlibrary.db")

5.33 s ± 168 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def get_distinct_values(group_var, dist_var, table_name, connect):
    sql_code = f"SELECT {group_var}, COUNT(DISTINCT({dist_var})) FROM {table_name} GROUP BY {group_var}"
    res = connect.execute(sql_code).fetchall()
    return res



def update_table(table_name, col_id, val, condition, connection):
    sql_code = f"UPDATE {table_name} SET {col_id} = '{val}' WHERE {condition}"
    connection.execute(sql_code)
    connection.commit()


connect = sqlite3.connect("genlibrary.db")
# count distinct SNPs for all Sample_id
res = get_distinct_values("Sample_ID", "SNP", "genstudio", connect)
print(res)
# update table meta - change sex for specific dna_chip_id
update_table("meta", "sex" , "Y", "dna_chip_id = '202341831127R04C02'", connect)
# check if updates are saved
sql_code = "SELECT * FROM meta WHERE sex = 'Y'"
res = connect.execute(sql_code).fetchall()
print(res)
connect.close()

[('202341831114R01C01', 8), ('202341831114R01C02', 8), ('202341831114R02C01', 8), ('202341831114R02C02', 8), ('202341831114R03C01', 8), ('202341831114R03C02', 8), ('202341831114R04C01', 8), ('202341831114R04C02', 8), ('202341831114R05C01', 8), ('202341831114R05C02', 8), ('202341831114R06C01', 8), ('202341831114R06C02', 8), ('202341831114R07C01', 8), ('202341831114R07C02', 8), ('202341831114R08C01', 8), ('202341831114R08C02', 8), ('202341831114R09C01', 8), ('202341831114R09C02', 8), ('202341831114R10C01', 8), ('202341831114R10C02', 8), ('202341831114R11C01', 8), ('202341831114R11C02', 8), ('202341831114R12C01', 8), ('202341831114R12C02', 8), ('202341831127R01C01', 8), ('202341831127R01C02', 8), ('202341831127R02C01', 8), ('202341831127R02C02', 8), ('202341831127R03C01', 8), ('202341831127R03C02', 8)]
[(840, '202341831127R04C02', 'Д', 'Y')]
